In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import numpy as np
import os
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

# Set paths for the dataset
train_dir = r'C:\Users\mahin\Downloads\dataset\train'
validation_dir = r'C:\Users\mahin\Downloads\dataset\validation'
test_dir = r'C:\Users\mahin\Downloads\dataset\test_images'

# Image data generators with improved augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True
)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

validation_generator = val_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)


Found 362 images belonging to 2 classes.
Found 14 images belonging to 2 classes.


In [6]:
# Load MobileNetV2 pre-trained on ImageNet
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers
base_model.trainable = False

# Add custom layers for classification
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, min_lr=1e-6)

# Train the model
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[early_stopping, reduce_lr]
)


C:\Users\mahin\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6704 - loss: 0.5881

C:\Users\mahin\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


12/12 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - accuracy: 0.6791 - loss: 0.5764 - val_accuracy: 1.0000 - val_loss: 0.1130 - learning_rate: 0.0010
Epoch 2/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.9098 - loss: 0.2297 - val_accuracy: 1.0000 - val_loss: 0.0572 - learning_rate: 0.0010
Epoch 3/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.9270 - loss: 0.1682 - val_accuracy: 1.0000 - val_loss: 0.0612 - learning_rate: 0.0010
Epoch 4/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.9217 - loss: 0.1551 - val_accuracy: 1.0000 - val_loss: 0.0508 - learning_rate: 0.0010
Epoch 5/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.9416 - loss: 0.1188 - val_accuracy: 1.0000 - val_loss: 0.0257 - learning_rate: 0.0010
Epoch 6/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 23s 2s/step - accuracy: 0.9615 - loss: 0.0880 - val_accuracy: 1.0000 - val_loss: 0.0459 - learning_rate: 0.0010
Epoch 7/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - accuracy: 0.9602 - loss: 0.0955 - val_accuracy: 1.0000 - va

In [ ]:
# Unfreeze some layers of the base model for fine-tuning
base_model.trainable = True

# Fine-tune with a lower learning rate
model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

history_finetune = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[early_stopping, reduce_lr]
)


Epoch 1/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 76s 4s/step - accuracy: 0.6334 - loss: 0.9940 - val_accuracy: 1.0000 - val_loss: 0.0242 - learning_rate: 1.0000e-05
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 44s 4s/step - accuracy: 0.6759 - loss: 0.7263 - val_accuracy: 1.0000 - val_loss: 0.0232 - learning_rate: 1.0000e-05
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 44s 4s/step - accuracy: 0.7551 - loss: 0.5423 - val_accuracy: 1.0000 - val_loss: 0.0218 - learning_rate: 1.0000e-05
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 43s 3s/step - accuracy: 0.8140 - loss: 0.4254 - val_accuracy: 1.0000 - val_loss: 0.0212 - learning_rate: 1.0000e-05
Epoch 5/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 45s 4s/step - accuracy: 0.8479 - loss: 0.3760 - val_accuracy: 1.0000 - val_loss: 0.0206 - learning_rate: 1.0000e-05
Epoch 6/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 44s 4s/step - accuracy: 0.8883 - loss: 0.2706 - val_accuracy: 1.0000 - val_loss: 0.0194 - learning_rate: 1.0000e-05
Epoch 7/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 47s 4s/step - accuracy: 0.8826 - loss:

In [ ]:
# Save the final model
model.save('cooked_vs_uncooked_advanced_model.keras')

# Load the saved model
best_model = tf.keras.models.load_model('cooked_vs_uncooked_advanced_model.keras')


In [ ]:
# Function for predicting a single image with test-time augmentation
def predict_image(img_path, model, augmentations=5):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0  # Rescale image

    predictions = []
    for _ in range(augmentations):
        augmented_img = test_datagen.random_transform(img_array[0])
        augmented_img = np.expand_dims(augmented_img, axis=0)
        prediction = model.predict(augmented_img, verbose=0)
        predictions.append(prediction[0][0])
    
    avg_prediction = np.mean(predictions)
    return 'uncooked' if avg_prediction > 0.5 else 'cooked'

# Predict images in the test directory
file_names = os.listdir(test_dir)
for file_name in file_names:
    img_path = os.path.join(test_dir, file_name)
    prediction = predict_image(img_path, best_model)
    print(f"Image: {file_name} - Prediction: {prediction}")
